In [1]:
import sys
sys.path.append('..')

from automl import AutoML, ProblemClassifier

import pandas as pd
import numpy as np

import sys, os
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

In [2]:
target = 'shot_made_flag'
id_column = 'shot_id'
df = pd.read_csv('data.csv').drop('game_date', axis=1)
df.shape

(30697, 24)

In [3]:
df, df_test = df[~np.isnan(df[target])], df[np.isnan(df[target])]
df.shape, df_test.shape

((25697, 24), (5000, 24))

In [4]:
X, y = df.drop([target, id_column], axis=1), df[target]

In [5]:
auto_ml = AutoML(problem_type=ProblemClassifier.CLASSIFICATION)

In [6]:
model = auto_ml.fit(X, y)

In [7]:
model.describe()

Preprocessing pipeline:
	Constant columns drop:
		Constant columns: ['team_id', 'team_name']
	Scaler:
		MinMaxScaler(copy=True, feature_range=(0, 1))
	Categorical converter:
		Ordinal: [], Nominal: ['combined_shot_type', 'shot_type', 'shot_zone_area', 'action_type', 'season', 'shot_zone_basic', 'opponent', 'shot_zone_range', 'matchup']
Model:
	LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


In [8]:
predictions = model.predict(df_test.drop([target, id_column], axis=1))

In [9]:
df_test.assign(**{target: predictions})[[id_column, target]].to_csv('submition.csv', index=False)